# 

In [1]:
# import the dataset

import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('data/us_pres_climate_speeches_final.db')
cursor = conn.cursor() # Create a cursor object

# Select all data from the table
query = f'SELECT * FROM {'us_pres_climate_speeches_final'};'
cursor.execute(query)

# Fetch all the data and store it in a pandas DataFrame
df_speeches_ori = pd.read_sql_query(query, conn)

conn.close()




In [2]:
df_speeches = df_speeches_ori.copy()

In [3]:
# Some adjusts...

# Convert column names to lowercase
df_speeches.columns = [col.lower() for col in df_speeches.columns] 

# Create a variable with party
party_mapping = {
    'George W. Bush': 'Republican',
    'Barack Obama': 'Democratic',
    'Donald Trump': 'Republican',
    'Joe Biden': 'Democratic'
}

# Create a new column 'party' based on the mapping
df_speeches['party'] = df_speeches['speaker'].map(party_mapping)

#
# Convert the 'date' column to datetime
df_speeches['date'] = pd.to_datetime(df_speeches['date'], format='%d/%m/%Y')


In [5]:
# Tidying data
from nltk.corpus import stopwords


# get lowercase tweets
df_speeches['cleaned_speech'] = df_speeches['text'].str.lower()

# Remove special characters, numbers, and symbols
df_speeches['cleaned_speech'] = df_speeches['cleaned_speech'].replace(to_replace=r'[^a-zA-Z0-9\s]', value='', regex=True) # drop punctuation and special char
df_speeches['cleaned_speech'] = df_speeches['cleaned_speech'].replace(to_replace=r'\b\d+\b', value='', regex=True) # drop numbers
#df_speeches['cleaned_speeches_with_SW'] = df_speeches['lowercase_speeches'].str.replace('[^\\w\\s]', '').replace(to_replace=r'\d', value='', regex=True)

# Remove stop words
stop_words = set(stopwords.words('english')) # get the list of stopwords

def remove_stop_words(text): #define a function to remove stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words] # compare each word with the list of stopwords
    return ' '.join(filtered_words)

df_speeches['cleaned_speech'] = df_speeches['cleaned_speech'].apply(remove_stop_words) # apply the function to each row

#drop some unnecessary columns
columns_to_drop = ["url", "title"]
df = df_speeches.drop(columns=columns_to_drop)
# 

In [7]:
# stemming and tokenization
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Lemmatization/Stemming
stemmer = PorterStemmer()

# Define a function for stemming
def stem_text(text):
    stemmed_words = [stemmer.stem(word) for word in text.split()] # Split the text into words and apply stemming to each word
    return ' '.join(stemmed_words) # Join the stemmed words back into a sentence
    
# Apply the stem_text function to each cleaned speech
df['stemmed_speech'] = df['cleaned_speech'].apply(stem_text)



# Tokenization
# Define a function
def tokenize_text(text):
    return word_tokenize(text)


df['tokens'] = df['stemmed_speech'].apply(tokenize_text)  #apply the function


In [10]:
df.head()

,date,speaker,text,party,cleaned_speech,stemmed_speech,tokens
0,2010-11-14,Barack Obama,"In 1994, APEC Leaders gathered in Bogor, Ind...",Democratic,apec leaders gathered bogor indonesia announce...,apec leader gather bogor indonesia announc sha...,"[apec, leader, gather, bogor, indonesia, annou..."
1,2010-11-20,Barack Obama,"20 November 2010, Lisbon We, the leaders of ...",Democratic,november lisbon leaders european union united ...,novemb lisbon leader european union unit state...,"[novemb, lisbon, leader, european, union, unit..."
2,2013-06-18,Barack Obama,2013 LOUGH ERNE Preamble 1. As leaders of th...,Democratic,lough erne preamble leaders g8 committed open ...,lough ern preambl leader g8 commit open econom...,"[lough, ern, preambl, leader, g8, commit, open..."
3,2015-06-08,Barack Obama,"Schloss Elmau, Germany We, the leaders of th...",Democratic,schloss elmau germany leaders g7 met elmau ann...,schloss elmau germani leader g7 met elmau annu...,"[schloss, elmau, germani, leader, g7, met, elm..."
4,2014-11-16,Barack Obama,1. Raising global growth to deliver better l...,Democratic,raising global growth deliver better living st...,rais global growth deliv better live standard ...,"[rais, global, growth, deliv, better, live, st..."


In [9]:
# export the cleanned file

df.to_csv('data/cleanned_df.csv', index=False)